In [12]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image

In [44]:
PATH_CSV_LABELS = 'tabellaPseudoLabels.csv'
PATH_DATASET_LAVORATO = './Dati Sensori/Dataset Sensori Lavorato'
PATH_MODELLO = 'Modelli/modello_pesi_inizializzati_v1'
PATH_MODELLO_FINALE = 'Modelli/modello_finale_v1'
PATH_MODELLO_FINALE_RIADDESTRATO = 'Modelli/modello_finale_riaddestrato_v1'

N_IMG = 1
FORMATO_IMG = 'jpg'
CHANNELS = 1
HEIGHT = 512
WIDTH = 512
TARGET_SIZE = (WIDTH, HEIGHT)

HUBER_DELTA = 1.0

In [3]:
df_labels = pd.read_csv(PATH_CSV_LABELS)
df_labels.head(5)

,Acquisizione,Label,Label_norm
0,12_M_1,0.558931,14.387763
1,12_M_2,0.848155,90.057848
2,12_M_3,0.886156,100.000000
3,12_M_4,0.771945,70.119024
4,12_M_5,0.744003,62.808388


In [4]:
def get_id_soggetto(id_acquisizione):
    id_acquisizione = id_acquisizione.split('_')
    n_soggetto = id_acquisizione[0]
    sesso_soggetto = id_acquisizione[1]
    return f"{n_soggetto}_{sesso_soggetto}"

In [5]:
X = []
y = []

for id_acquisizione in df_labels.Acquisizione:
    nome_file = f"{id_acquisizione}_img_{N_IMG}.{FORMATO_IMG}"
    path_img = os.path.join(PATH_DATASET_LAVORATO, get_id_soggetto(id_acquisizione), id_acquisizione, nome_file)
    image = Image.open(path_img)
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    
    pseudo_label = df_labels[df_labels.Acquisizione == id_acquisizione].Label_norm
    
    X.append(normalized_image_array)
    y.append(pseudo_label)
    
X = np.array(X)
y = np.array(y)

In [6]:
model = keras.models.load_model(PATH_MODELLO)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 510, 510, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 255, 255, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 2080800)           0         
                                                                 
 dense (Dense)               (None, 64)                133171264 
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 133,171,649
Trainable params: 133,171,649
Non-trainable params: 0
__________________________________________

In [7]:
model.fit(x=X, y=y, batch_size=1, epochs=20)

Epoch 1/20
30/30 [==============================] - 15s 470ms/step - loss: 26.7575 - mae: 26.7575
Epoch 2/20
30/30 [==============================] - 14s 471ms/step - loss: 22.8952 - mae: 22.8952
Epoch 3/20
30/30 [==============================] - 14s 469ms/step - loss: 22.2681 - mae: 22.2681
Epoch 4/20
30/30 [==============================] - 14s 470ms/step - loss: 21.7695 - mae: 21.7695
Epoch 5/20
30/30 [==============================] - 14s 470ms/step - loss: 20.6866 - mae: 20.6866
Epoch 6/20
30/30 [==============================] - 14s 465ms/step - loss: 19.9276 - mae: 19.9276
Epoch 7/20
30/30 [==============================] - 14s 471ms/step - loss: 19.0098 - mae: 19.0098
Epoch 8/20
30/30 [==============================] - 14s 472ms/step - loss: 17.8099 - mae: 17.8099
Epoch 9/20
30/30 [==============================] - 14s 472ms/step - loss: 16.5673 - mae: 16.5673
Epoch 10/20
30/30 [==============================] - 14s 469ms/step - loss: 13.6656 - mae: 13.6656
Epoch 11/20
30/30 [

In [8]:
model.predict(X)

1/1 [==============================] - 1s 569ms/step


array([[13.310622 ],
       [86.78608  ],
       [97.427986 ],
       [67.046776 ],
       [56.174347 ],
       [ 5.7214327],
       [67.83462  ],
       [48.542877 ],
       [70.3464   ],
       [64.93754  ],
       [29.97097  ],
       [55.259003 ],
       [73.60178  ],
       [10.185326 ],
       [54.465794 ],
       [32.542233 ],
       [33.75649  ],
       [61.59501  ],
       [65.85577  ],
       [62.02124  ],
       [29.181412 ],
       [43.423046 ],
       [40.91028  ],
       [51.24464  ],
       [72.74597  ],
       [19.589087 ],
       [18.941109 ],
       [ 8.066226 ],
       [39.02061  ],
       [24.834421 ]], dtype=float32)

In [9]:
y

array([[ 14.38776324],
       [ 90.05784833],
       [100.        ],
       [ 70.11902421],
       [ 62.8083878 ],
       [  0.        ],
       [ 75.6853248 ],
       [ 53.78656739],
       [ 73.88803792],
       [ 69.96671962],
       [ 31.43078081],
       [ 61.84041709],
       [ 73.70590257],
       [  1.90248829],
       [ 61.10732865],
       [ 31.94502288],
       [ 36.98859557],
       [ 64.10041713],
       [ 71.95538621],
       [ 62.96287593],
       [ 33.46659409],
       [ 45.18621211],
       [ 48.7416397 ],
       [ 54.43098089],
       [ 75.04505273],
       [ 21.64331626],
       [ 25.37730258],
       [  6.16541405],
       [ 41.17895281],
       [ 24.97404123]])

In [11]:
model.save(PATH_MODELLO_FINALE)

INFO:tensorflow:Assets written to: Modelli/modello_finale_v1/assets


INFO:tensorflow:Assets written to: Modelli/modello_finale_v1/assets


## Addestramento nuovo modello con Huber loss

In [36]:
conv1_weights = model.layers[0].weights
dense1_weights = model.layers[3].weights
dense2_weights = model.layers[4].weights

new_model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS), weights=conv1_weights),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu', weights=dense1_weights),
    layers.Dense(1, weights=dense2_weights)
])

In [39]:
new_model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=HUBER_DELTA), metrics=['mae'])

In [40]:
new_model.fit(x=X, y=y, batch_size=1, epochs=20)

Epoch 1/20
30/30 [==============================] - 36s 1s/step - loss: 2.5977 - mae: 3.0546
Epoch 2/20
30/30 [==============================] - 35s 1s/step - loss: 2.2721 - mae: 2.7003
Epoch 3/20
30/30 [==============================] - 35s 1s/step - loss: 2.4918 - mae: 2.9762
Epoch 4/20
30/30 [==============================] - 35s 1s/step - loss: 2.6903 - mae: 3.1549
Epoch 5/20
30/30 [==============================] - 35s 1s/step - loss: 0.7914 - mae: 1.2215
Epoch 6/20
30/30 [==============================] - 36s 1s/step - loss: 1.1671 - mae: 1.5990
Epoch 7/20
30/30 [==============================] - 36s 1s/step - loss: 0.8600 - mae: 1.2721
Epoch 8/20
30/30 [==============================] - 37s 1s/step - loss: 0.7616 - mae: 1.1599
Epoch 9/20
30/30 [==============================] - 36s 1s/step - loss: 1.1982 - mae: 1.6247
Epoch 10/20
30/30 [==============================] - 36s 1s/step - loss: 2.1240 - mae: 2.5760
Epoch 11/20
30/30 [==============================] - 37s 1s/step - lo

In [41]:
new_model.predict(X)

1/1 [==============================] - 1s 1s/step


array([[15.588796 ],
       [86.49542  ],
       [96.01784  ],
       [69.962494 ],
       [59.4548   ],
       [ 2.271685 ],
       [76.13985  ],
       [51.673805 ],
       [72.6875   ],
       [66.72004  ],
       [32.07514  ],
       [61.118534 ],
       [70.90638  ],
       [ 2.271685 ],
       [59.40814  ],
       [30.268415 ],
       [35.656864 ],
       [62.4236   ],
       [69.06589  ],
       [60.893192 ],
       [32.62777  ],
       [44.50001  ],
       [47.18455  ],
       [54.385796 ],
       [72.56381  ],
       [19.149496 ],
       [26.162617 ],
       [ 4.6359863],
       [37.129784 ],
       [24.908323 ]], dtype=float32)

In [42]:
y

array([[ 14.38776324],
       [ 90.05784833],
       [100.        ],
       [ 70.11902421],
       [ 62.8083878 ],
       [  0.        ],
       [ 75.6853248 ],
       [ 53.78656739],
       [ 73.88803792],
       [ 69.96671962],
       [ 31.43078081],
       [ 61.84041709],
       [ 73.70590257],
       [  1.90248829],
       [ 61.10732865],
       [ 31.94502288],
       [ 36.98859557],
       [ 64.10041713],
       [ 71.95538621],
       [ 62.96287593],
       [ 33.46659409],
       [ 45.18621211],
       [ 48.7416397 ],
       [ 54.43098089],
       [ 75.04505273],
       [ 21.64331626],
       [ 25.37730258],
       [  6.16541405],
       [ 41.17895281],
       [ 24.97404123]])

In [45]:
new_model.save(PATH_MODELLO_FINALE_RIADDESTRATO)

INFO:tensorflow:Assets written to: Modelli/modello_finale_riaddestrato_v1/assets


INFO:tensorflow:Assets written to: Modelli/modello_finale_riaddestrato_v1/assets
